In [1]:
from pytube import YouTube
import numpy as np
import pandas as pd
import json
from pyyoutube import Api
import requests
import nltk
# from nltk.corpus import wordnet
# nltk.download('wordnet')

In [65]:
# data = json.load(open("research/input/annotation.json"))
# del data["wLXBWt7skfM"]
data0 = json.load(open("research/input/strategy_vids_0.json"))
data1 = json.load(open("research/input/strategy_vids_1.json"))
data2 = json.load(open("research/input/strategy_vids_2.json"))

In [66]:
data = {}
for d in (data0, data1, data2): 
    data.update(d)

In [51]:
data = json.load(open("research/input/new_annotations.json"))

In [12]:
for video_tag, annots in data.items():
    print(video_tag, annots)
    break

ZNz2qF6r3Kc {'url': 'https://www.youtube.com/watch?v=ZNz2qF6r3Kc', 'topics': ['Reciprocity, Social Impact, Social Proof']}


In [15]:
for video_tag, annots in data.items():
    print(video_tag, annots['topics'])
    break

ZNz2qF6r3Kc ['Reciprocity, Social Impact, Social Proof']


In [16]:
API_KEY = json.load(open("research/input/creds.json"))["api_key"]
api = Api(api_key=API_KEY)

### private videos

In [17]:
not_working = {}
for video_tag, annots in data.items():
    link = "https://www.youtube.com/watch?v=" + video_tag
    video = YouTube(link)
    try:
        video.check_availability()
    except Exception as e:
        not_working[video_tag] = 0


KeyboardInterrupt



In [18]:
print(not_working)

{}


### Youtube Data 

In [69]:
companies = {}
j = 0
youtube_data = {}
for video_tag, annots in data.items():
    j+=1
    if j%1000 == 0:
        print(j)
    if video_tag not in not_working:
        if len(api.get_video_by_id(video_id=video_tag).items) > 0:
            video = api.get_video_by_id(video_id=video_tag).items[0].to_dict()
        else:
            continue
        youtube_data[video_tag] = {}
        youtube_data[video_tag]["annots"] = annots
        youtube_data[video_tag]["publishedAt"] = video["snippet"]["publishedAt"]
        youtube_data[video_tag]["title"] = video["snippet"]["title"]
        youtube_data[video_tag]["description"] = video["snippet"]["description"]
        channel = video["snippet"]["channelTitle"]
        channelID = video["snippet"]["channelId"]
        youtube_data[video_tag]["channelTitle"] = channel
        youtube_data[video_tag]["channelId"] = channelID
        youtube_data[video_tag]["tags"] = video["snippet"]["tags"]
        youtube_data[video_tag]["viewCount"] = int(video["statistics"]["viewCount"])
        
        youtube_data[video_tag]["likeCount"] = int(video["statistics"]["likeCount"]) if video["statistics"]["likeCount"] != None else 0
        # youtube_data[video_tag]["dislikeCount"] = int(video["statistics"]["dislikeCount"]) if video["statistics"]["dislikeCount"] != None else 0
        # youtube_data[video_tag]["commentCount"] = video["statistics"]["commentCount"] if video["statistics"]["commentCount"] != None else 0
        if "commentCount" in video["statistics"] and video["statistics"]["commentCount"] is not None:
            youtube_data[video_tag]["commentCount"] = int(video["statistics"]["commentCount"])
        else:
            youtube_data[video_tag]["commentCount"] = 0
        youtube_data[video_tag]["topicCategories"] = video["topicDetails"]["topicCategories"] if video["topicDetails"] != None else []

        if channel not in companies:
            companies[channel] = [channelID,[], 0]
        companies[channel][1].append(video_tag)
        companies[channel][2] += 1

1000
2000
3000
4000


PyYouTubeException: YouTubeException(status_code=403,message=The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.)

In [35]:
# "8xVeJDwpeNo"
limit = 5

for index, (video_tag, annots) in enumerate(data.items()):
    video = api.get_video_by_id(video_id=video_tag).items[0].to_dict()
    print(youtube_data[video_tag]["commentCount"])
    # print(video["statistics"])
    if index >= limit:
        break

# for video_tag, annots in data.items():
#     video = api.get_video_by_id(video_id=video_tag).items[0].to_dict()
#     print(video["statistics"])
#     break

0
0
0
39
2
78


### sectors

In [36]:
yt_channel_to_company_mappings = {
    "stanley tools":"Stanley Black & Decker",
    "o'reilly auto parts":"O'Reilly Automotive",
    "southern california edison":"Edison International",
    "wd":"Western Digital",
    "viacom18 studios": "Viacom",
    "corning incorporated": "Corning",
    "bd": "Becton Dickinson",
    "hii": "Huntington Ingalls",
    "new york life insurance company": "New York Life Insurance",
    "pacific gas & electric": "PG&E",
    "cigna healthcare" : "Cigna", 
    "progressive insurance": "Progressive",
    "ups": "United Parcel Service",
    "caci international inc": "CACI International",
    "consumers energy": "CMS Energy",
    "microntechnology": "Micron Technology",
    "kenworth truck co.": "Paccar"
}

In [37]:
fortune500 = pd.read_csv("research/input/fortune-500.csv")

In [38]:
sectors = {}
_500companies = {}
for i in range(len(fortune500)):
    company = fortune500["Name"].iloc[i]
    sector = fortune500["Sector"].iloc[i]
    if sector not in sectors:
        sectors[sector] = 0
    _500companies[company.lower()] = sector
    sectors[sector] += 1

In [39]:
def add_sector(dict_, company_entry, sector):
    for entry in company_entry[1]:
        dict_[entry]["sector"] = sector
    return dict_

In [40]:
company_list = sorted(list(_500companies.keys()))
company_vocab = {}
for company in company_list:
    tokens = company.split()
    for token in tokens:
        if token not in company_vocab:
            company_vocab[token] = []
        company_vocab[token].append(company)

In [41]:
import difflib as dl

j = 0
for company, values in companies.items():
    company = company.lower()
    j+=1
    if company not in _500companies:
        flag = 1
        for _500company in company_list:
            if company in _500company:
                youtube_data = add_sector(youtube_data, values, _500companies[_500company])
                flag = 0
                break
                
        if flag == 1:
            possible = dl.get_close_matches(company, company_list)
            if len(possible)>0:
                if len(possible) == 1:
                    youtube_data = add_sector(youtube_data, values, _500companies[possible[0]])
                    flag = 0
        if flag == 1:
            split_names = company.split()
            buffer = []
            for split_name in split_names:
                if split_name in company_vocab:
                    buffer += company_vocab[split_name]
                else:
                    pass
            if len(buffer) == 1:
                youtube_data = add_sector(youtube_data, values, _500companies[buffer[0]])
            else:
                youtube_data = add_sector(youtube_data, values, _500companies[yt_channel_to_company_mappings[company].lower()])
    else:
        youtube_data = add_sector(youtube_data, values, _500companies[company])

In [42]:
with open('research/output/youtube_data.json', 'w') as f:
    json.dump(youtube_data, f)

### transcript and comments